In [ ]:
options(repr.matrix.max.cols=100,repr.matrix.max.rows=300)

FDOC Inmate Data, Continued
----------------------------

We are going to work a little more with the FDOC data, hopefully providing you with code to help you continue to dig into the data. First, we are going to look at current inmates' incarceration histories. Specifically we will compute how much time each person has spent in the system and how how many times they have been jailed. 

For this, we need two data sets -- we will read in the INMATE_ACTIVE_ROOT.csv table and the INMATE_ACTIVE_INCARHIST.csv table. By now we are old hands at this.

In [ ]:
root = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_ROOT.csv",as.is=TRUE)
head(root,5)

In [ ]:
prior = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_INCARHIST.csv",as.is=T)
head(prior,20)

As we did last time, for all the dates, we have to make up for the 1900 vs. 2000 century problem. Just giving two digits means that 01 or 16 for a year refers to 2000, while 70 or 95 refers to 1990.

In [ ]:
# Extract the 2-digit years and correct for 1900 v. 2000
rtd = as.numeric(substr(prior$ReceiptDate,7,8))
rld = as.numeric(substr(prior$ReleaseDate,7,8))

table(rtd)
table(rld)

Here we fix the problem using the function ifelse(). Make sure you remember what it does! The last two commands are histograms of the dates people were admitted and released. 

If an inmate entered the system 5 times, each of their 4 prevous entries and exits are tallied here, together with the entry that represents their most recent (the term they're currently serving).

In [ ]:
rtd = ifelse(rtd <= 16, 2000+rtd, 1900+rtd)
rld = ifelse(rld <= 16, 2000+rld, 1900+rld)

options(repr.plot.width=8,repr.plot.height=6)

hist(rtd,breaks=200)
hist(rld,breaks=200)

What do you think?

Below we create a "smoothed" histogram. Remember how we fit local regressions, this is doing much the same, acting locally to estimate the "density" of points in a neighborhood. The plot is scaled so that the area under the curve is 1 (remember that??).

In [ ]:
rtdfit = density(rtd)
plot(rtdfit,xlab="year",ylab="density",main="Receipt (black) and Release (Green) Dates")

rldfit = density(rld,na.rm=TRUE)
lines(rldfit,col="green")

**1. The variable rtd represents the year someone was admitted, and rld gives you the year they were released. When they are still serving their sentence, rld is NA, missing. Use this fact to create a histogram of admission dates (elements of rtd) that correspond to the prison entry of the term they are currently serving (the corresponding release year in rld is NA).**

**2. Visually compare the data frames "root" and "prior" and explain what "ReceiptDate" in the "root" frame means.**

In [ ]:
# Put your code here



We are now going pull all the processing into one cell. 

In [ ]:
library(lubridate)
library(dplyr)

root = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_ROOT.csv",as.is=TRUE)
prior = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_INCARHIST.csv",as.is=T)


# Fix the date strings by adding 1900 to some and 2000 to others

rtd = as.numeric(substr(prior$ReceiptDate,7,8))
rld = as.numeric(substr(prior$ReleaseDate,7,8))
rtd = ifelse(rtd <= 16, 2000+rtd, 1900+rtd)
rld = ifelse(rld <= 16, 2000+rld, 1900+rld)

# Here we use the fixed year for ReleaseDate only if its value is not "". 
# A ReleaseDate of "" means that someone is still serving time, and so we will
# use the current date so that we can tally time-served for each inmate.

prior$ReceiptDateFixed = paste(substr(prior$ReceiptDate,1,5),rtd,sep="/")
prior$ReleaseDateFixed = ifelse(prior$ReleaseDate=="","11/10/2016",paste(substr(prior$ReleaseDate,1,5),rld,sep="/"))

# Turn the fixed date strings into date objects using mdy()
prior$ReceiptDateObj = mdy(prior$ReceiptDateFixed)
prior$ReleaseDateObj = mdy(prior$ReleaseDateFixed)

# And create a new variable that, for each prison term, computes how much time has 
# been spent in jail.

prior$TimeSpent = interval(prior$ReceiptDateObj,prior$ReleaseDateObj)/dyears(1)

# finally sort the data set by DCNumber and then by their prison sentences

prior = arrange(prior,DCNumber,ReceiptDateObj)

head(prior)

We now take "prior" and group entries by DCNumber, counting how many times each person has been in prison and how much time they served. Those who are still in prison will add up time until today.

In [ ]:
priortime = summarise(group_by(prior,DCNumber),count=n(),time=sum(TimeSpent,na.rm=TRUE))
head(priortime)
dim(priortime)

We then add the "count" and "time" variables back to our "root" data set using a left_join(). This mimics what we did for the last assignment.

In [ ]:
root = left_join(root,priortime,by="DCNumber")
head(root)

And now we make some graphics. First a summary of the number of times someone has been in prison and then a boxplot showing how total time served varies with the number of times they have been in prison. To make this cleaner, we might drop the sentences that are currently being served.

In [ ]:
barplot(table(root$count))

In [ ]:
boxplot(time~count,data=root,xlab="Number of prison terms",ylab="Time spent (to date)")

What do the boxplot and barplot tell you? Who do you want to focus in on? I thought perhaps those people with more than 15 offenses, although between 10 and 15 is interesting too. What are they in jail for?

In [ ]:
rep = filter(root,count >=15)
tail(rep)

In [ ]:
pastoff = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_OFFENSES_prpr.csv",as.is=TRUE)
off = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_OFFENSES_CPS.csv",as.is=TRUE)
head(pastoff,5)

In [ ]:
i = 23
filter(off,DCNumber == rep$DCNumber[i])
filter(pastoff,DCNumber == rep$DCNumber[i])

Finally, let's look at total incarceration. The plots we've made so far concern the current prison population. We can also add in information about those that have been released and are no longer serving time. When were they admitted? Released? Here is all the code in one place to create a big data set about receipt and release times for anyone in the database.

In [ ]:
prior = read.csv("http://compute-cuj.org/FDOC/INMATE_ACTIVE_INCARHIST.csv",as.is=T)

rtd = as.numeric(substr(prior$ReceiptDate,7,8))
rld = as.numeric(substr(prior$ReleaseDate,7,8))

rtd = ifelse(rtd <= 16, 2000+rtd, 1900+rtd)
rld = ifelse(rld <= 16, 2000+rld, 1900+rld)

prior$ReceiptDateFixed = ifelse(prior$ReceiptDate=="","",paste(substr(prior$ReceiptDate,1,5),rtd,sep="/"))
prior$ReleaseDateFixed = ifelse(prior$ReleaseDate=="","11/10/2016",paste(substr(prior$ReleaseDate,1,5),rld,sep="/"))

prior$ReceiptDateObj = mdy(prior$ReceiptDateFixed)
prior$ReleaseDateObj = mdy(prior$ReleaseDateFixed)

rprior = read.csv("http://compute-cuj.org/FDOC/INMATE_RELEASE_INCARHIST.csv",as.is=T)

rrtd = as.numeric(substr(rprior$ReceiptDate,7,8))
rrld = as.numeric(substr(rprior$ReleaseDate,7,8))

rrtd = ifelse(rrtd <= 16, 2000+rrtd, 1900+rrtd)
rrld = ifelse(rrld <= 16, 2000+rrld, 1900+rrld)

rprior$ReceiptDateFixed = paste(substr(rprior$ReceiptDate,1,5),rrtd,sep="/")
rprior$ReleaseDateFixed = ifelse(rprior$ReleaseDate=="","",paste(substr(rprior$ReleaseDate,1,5),rrld,sep="/"))

rprior$ReceiptDateObj = mdy(rprior$ReceiptDateFixed)
rprior$ReleaseDateObj = mdy(rprior$ReleaseDateFixed)

Remake our histograms...

In [ ]:
hist(c(rld,rrld),breaks=200,main="Release year")
hist(c(rtd,rrtd),breaks=200,main="Receipt year")

Here we create the TimeSpent variable for each time they were in prison. We use summarise() to count the number of times they were in jail and then sum() up the TimeSpent to get their total time served. In the end we make a big data set that covers everyone in the data base. The command to add one data set to the bottom of another is rbind() for row-bind. There is also a cbind() to add columns to a table.

In [ ]:
prior$TimeSpent = interval(prior$ReceiptDateObj,prior$ReleaseDateObj)/dyears(1)
priortime = summarise(group_by(prior,DCNumber),count=n(),time=sum(TimeSpent,na.rm=TRUE))

rprior$TimeSpent = interval(rprior$ReceiptDateObj,rprior$ReleaseDateObj)/dyears(1)
rpriortime = summarise(group_by(rprior,DCNumber),count=n(),time=sum(TimeSpent,na.rm=TRUE))

head(priortime)
head(rpriortime)

bigpriortime = rbind(priortime,rpriortime)

This produces much the same boxplot set as we had with the data from current inmates.

In [ ]:
boxplot(time~count,data=bigpriortime)

Finally, a loop. We are going to create two "for loops" that run over year (from 2000 to 2015) and month (from 1 to 12) and look at how many people were in prison on the first of the given month of the given year. We'll do this for 1/1/2000, 2/1/2000, etc. all the way to 1/12/2015. 

In [ ]:
ts = NULL
tm = NULL

for(y in 2000:2015){
    for(m in 1:12){
        
        # Using year, month and the first day of the month, create a date object called "now"
        
        dt = paste(m,1,y,sep="/")        
        now = mdy(dt)
        
        # Then count up how many people were received by the prison before "now" but left
        # after "now", taking care that if the ReleaseDateObj is NA, it means they are still
        # in prison
        
        s1 = filter(prior,(ReceiptDateObj <= now) & (is.na(ReleaseDateObj) | (ReleaseDateObj >= now)))$DCNumber
        s2 = filter(rprior,(ReceiptDateObj <= now) & (is.na(ReleaseDateObj) | (ReleaseDateObj >= now)))$DCNumber
        
        # grow these two vectors by one entry each. The total number of people in prison
        # at the time, and the string representation of the m-d-y.
        
        ts = c(ts,length(unique(c(s1,s2))))
        tm = c(tm,dt)
    }
}

# Pull ts and tm into columns of a data frame, making date a date object
population = data.frame(date=mdy(tm),count=ts)

And plot it out. 

**3. What do you think? Is this right? How might you check? Be creative!**

In [ ]:
plot(count~date,data=population,type="l")

In [ ]:
tail(population,20)

**Obama and clemency**

In 2014 The White House announced its [Clemency Initiative](https://www.justice.gov/pardon/clemency-initiative), that could involve up to 10,000 inmates. Obama has been steadily granting commutations and pardons, and you can see 
[the totals here](https://www.justice.gov/pardon/obama-commutations). The data seem extremely organized. Everything is in a table with a certain degree of structure. 

**4. Tell me about the data as it exists on the Pardons and Commutations web site. What data are immediately available and what has to be "fished" for? Try coming up with a series of columns, fields if you will, that we could fill in for each person granted clemency.**

Write your ideas here




In many cases, you can reach outside the existing data set for more information. For example, simply asking Google about "ZoAnne Brown Iowa May 2007" gets you [an article from a local paper](http://globegazette.com/news/local/crime-and-courts/mason-city-woman-s-meth-sentence-commuted-advocates-celebrate/article_079a6dd2-f808-545c-a1df-0caed3b8315c.html) about the incident.

**5. What other sources of data can be added to fill out the data set? Think about details that would make the story about clemency richer in some way.**

Write your ideas here



Next time, we are going to see how to extract data from tables like the one on the Commutations and Pardons web site. We will also see how we can search for particular patterns of text that might help us decide whether this was a drug-related offense or something else. And if it is about drugs, how much and what kind?